In [18]:
#importando librerias necesarias para trabajar el algoritmo genetico
import pandas as pd
import numpy as np
import time
import json
from datetime import datetime

In [19]:
def improved_ultra_fast_ga(dataset_file: str, max_minutes: int = 3):
    """
    Algoritmo genético ultra-rápido mejorado que supera consistentemente los puntos de referencia
    """
    print(f"\n--IMPLEMENTANDO ALGORITMO GENETICO ({max_minutes} min max)")

    df = pd.read_csv(dataset_file, encoding='utf-8')#carga del dataset

    # configuracion para rendimiento superior
    total_budget = df['MONTO_VIABLE'].sum() #suma todos los montos de proyectos
    config = {
        'max_budget': total_budget * 0.25,           # 25% del presupuesto total como limite
        'max_projects': min(len(df) // 8, 300),     # Más proyectos permitidos
        'population_size': 25,                       # Población más grande
        'generations': 50,                           # Más generaciones
        'crossover_rate': 0.8,                       # Mayor tasa de cruce
        'mutation_rate': 0.02,                       # Tasa de mutación más baja
        'elitism_size': 5,                           # Mantener los 5 mejores
        'time_limit_seconds': max_minutes * 60
    }

    # Mostrar configuración

    print(f"   --Configuración mejorada:")
    print(f"   Projectos: {len(df):,}")
    print(f"   Presupuesto máximo: ${config['max_budget']:,.0f}")
    print(f"   Proyectos máximos: {config['max_projects']}")
    print(f"   Población: {config['population_size']}")
    print(f"   Generationes: {config['generations']}")

    # Pre-compute metrics for speed
    costs = df['MONTO_VIABLE'].values           #combierte array numpy para velocidad
    progress = df['AVANCE_FISICO'].values       #array con porcentaje de avance fisico
    beneficiaries = df['BENEFICIARIO'].values   #array con beneficiarios

    # Calculate efficiency scores
    efficiency = np.where(costs > 0, progress / (costs / 1_000_000), 0) #calcula eficiencia si costo>0 , eficiencia = avance fisico / (costo/1_000_000)
    efficiency = np.nan_to_num(efficiency, nan=0, posinf=1000, neginf=0) # Replaces NaN, inf, -inf with 0, 1000, 0 respectively

    #funcion para crear individuos inteligentes
    def create_smart_individual():
        """Create individual with bias toward efficient projects"""
        individual = np.zeros(len(df), dtype=np.int8) #crear array de ceros usando entero de 8 bits

        # crear pesos de probabilidad basados en eficiencia
        weights = efficiency / (efficiency.sum() + 1e-10) #normaliza eficiencia a probabilidades
        weights = np.maximum(weights, 0.001)  # asegura probabilidades minimas para todos

        #seleccion inteligente de proyectos
        current_budget = 0 #presupuesto usado inicialmente
        selected = 0 #numero de proyectos seleccionados
        tried_indices = set() #indices de proyectos ya probados

        # Seleccionar proyectos hasta alcanzar el maximo de proyectos o presupuesto
        while selected < config['max_projects'] and len(tried_indices) < len(df):

            # Seleccionar indice basado en pesos de eficiencia
            available_indices = [i for i in range(len(df)) if i not in tried_indices] #indices disponibles
            if not available_indices:
                break

            available_weights = weights[available_indices] #pesos de eficiencia de proyectos disponibles
            available_weights = available_weights / available_weights.sum()# normaliza pesos disponibles

            idx = np.random.choice(available_indices, p=available_weights)#selecciona indice basado en pesos
            tried_indices.add(idx) #marcamos como probado

            # Verificar si el proyecto cabe en el presupuesto
            if current_budget + costs[idx] <= config['max_budget']:
                individual[idx] = 1  #selecciona el proyecto
                current_budget += costs[idx] #actualiza presupuesto
                selected += 1 #incrementa proyectos seleccionados
        return individual #se retorna el individuo creado

    #funcion de fitness mejorada
    def enhanced_fitness(individual):
        """Funcion de fitness mejorada con multiples objetivos"""
        #si no hay proyecto seleccionado, retorna 0
        if individual.sum() == 0:
            return 0

        selected = individual.astype(bool) #convierte el individuo a booleano para seleccionar proyectos

        # primer objetivo: progreso total
        total_progress = progress[selected].sum() #suma de avance fisico de proyectos seleccionados
        total_cost = costs[selected].sum() #suma de costos de proyectos seleccionados
        total_beneficiaries = beneficiaries[selected].sum()#suma de beneficiarios de proyectos seleccionados

        # penalizacion por presupuesto (cuadratica para violaciones)
        if total_cost > config['max_budget']:
            budget_penalty = ((total_cost - config['max_budget']) / config['max_budget']) ** 2 # penalizacion cuadratica
            total_progress -= budget_penalty * 2000 # penalizacion por presupuesto violado, resta la penalizacion del fitness

        # Bono por eficiencia
        if total_cost > 0:
            avg_efficiency = np.mean(efficiency[selected]) #eficiencia promedio de proyectos seleccionados
            efficiency_bonus = min(avg_efficiency / 10, 100) # bono por eficiencia, maximo 100
            total_progress += efficiency_bonus # se suma el bono por eficiencia

        # Bono por impacto social
        if total_beneficiaries > 0:
            social_bonus = min(total_beneficiaries / 10000, 50) # bono por impacto social, maximo 50
            total_progress += social_bonus

        # Bono por diversidad (departamentos cubiertos)
        dept_diversity = df.loc[selected, 'DEPARTAMENTO'].nunique() #departamentos únicos cubiertos por proyectos seleccionados
        diversity_bonus = dept_diversity * 2 # bono por diversidad, 2 puntos por departamento único
        total_progress += diversity_bonus

        return max(0, total_progress) #asegura que el fitness no sea negativo

    # Funciones de cruce y mutación inteligentes
    def smart_crossover(parent1, parent2):

        """Cruce inteligente que preserva buenos genes"""

        if np.random.random() > config['crossover_rate']: #si no se cruza, retorna copias de los padres
            return parent1.copy(), parent2.copy()

        # Cruce de dos puntos
        points = sorted(np.random.choice(len(df), 2, replace=False)) #selecciona dos puntos ordenados aleatorios

        child1 = parent1.copy() #copia del padre 1
        child2 = parent2.copy() #copia del padre 2

        # Intercambiar sección media
        child1[points[0]:points[1]] = parent2[points[0]:points[1]] # Intercambiar sección media
        child2[points[0]:points[1]] = parent1[points[0]:points[1]] # Intercambiar sección media

        return repair_individual(child1), repair_individual(child2) #reparar individuos para cumplir restricciones

    # Mutación inteligente
    def smart_mutation(individual):
        """Mutación inteligente con intercambios basados en eficiencia"""
        individual = individual.copy() # crea una copia del individuo para no modificar el original

        # Inversiones de bits aleatorias
        for i in range(len(individual)): #iterar sobre cada proyecto
            if np.random.random() < config['mutation_rate']: #probabilidad de mutación
                individual[i] = 1 - individual[i] #invertir el bit (seleccionado/desseleccionado)

        # Intercambio inteligente (reemplazar el peor con el mejor disponible)
        if np.random.random() < 0.3:  # 30% de probabilidad
            selected_indices = np.where(individual)[0] #indices de proyectos seleccionados
            unselected_indices = np.where(individual == 0)[0] #indices de proyectos no seleccionados

            if len(selected_indices) > 0 and len(unselected_indices) > 0:
                # Encontrar el peor proyecto seleccionado
                selected_efficiencies = efficiency[selected_indices]#eficiencia de proyectos seleccionados
                worst_idx = selected_indices[np.argmin(selected_efficiencies)]# menor eficiencia entre proyectos seleccionados

                # Encontrar el mejor proyecto no seleccionado que se ajuste al presupuesto
                current_cost = costs[selected_indices].sum()# costo total de proyectos seleccionados
                worst_cost = costs[worst_idx]# costo del peor proyecto seleccionado
                available_budget = config['max_budget'] - current_cost + worst_cost # presupuesto disponible si se quita el peor proyecto seleccionado

                feasible_unselected = unselected_indices[costs[unselected_indices] <= available_budget]# proyectos no seleccionados que caben en el presupuesto disponible

                if len(feasible_unselected) > 0:
                    feasible_efficiencies = efficiency[feasible_unselected]# eficiencias de proyectos no seleccionados que caben en el presupuesto
                    best_idx = feasible_unselected[np.argmax(feasible_efficiencies)]# mejor eficiencia entre proyectos no seleccionados que caben en el presupuesto

                    # Swap
                    individual[worst_idx] = 0 # quitar el peor proyecto seleccionado
                    individual[best_idx] = 1 # añadir el mejor proyecto no seleccionado

        return repair_individual(individual)

    def repair_individual(individual):
        """Repara el individuo para cumplir las restricciones"""
        individual = individual.copy()
        selected_indices = np.where(individual)[0] #indices de proyectos seleccionados

        if len(selected_indices) == 0:
            return individual #si no hay proyectos seleccionados, retorna el individuo sin cambios

        # reparar presupuesto - eliminar el más caro hasta que sea factible
        while len(selected_indices) > 0:
            total_cost = costs[selected_indices].sum() #costo total de proyectos seleccionados

            if total_cost <= config['max_budget'] and len(selected_indices) <= config['max_projects']:
                break

            # eliminar el proyecto menos eficiente
            selected_efficiencies = efficiency[selected_indices] #eficiencia de proyectos seleccionados
            worst_idx = selected_indices[np.argmin(selected_efficiencies)] # encontrar el menos eficiente
            individual[worst_idx] = 0 # quitar el proyecto menos eficiente
            selected_indices = np.where(individual)[0] # actualizar indices de proyectos seleccionados

        return individual

    #seleccion por torneo
    def tournament_selection(population, fitness_scores, tournament_size=3): # population: lista de individuos, fitness_scores: lista de puntajes de fitness
        """        Selección por torneo para elegir padres"""
        tournament_indices = np.random.choice(len(population), tournament_size, replace=False) #Selecciona 3 individuos aleatoriamente sin reemplazo
        tournament_fitness = [fitness_scores[i] for i in tournament_indices] # calcular fitness de los individuos seleccionados
        winner_idx = tournament_indices[np.argmax(tournament_fitness)] # encontrar el índice del mejor individuo del torneo
        return population[winner_idx].copy() #retorna una copia del mejor individuo seleccionado

    # Bucle principal del algoritmo genético
    start_time = time.time() # marca el tiempo de inicio

    # Crear población inicial con individuos inteligentes
    print("-- Creando una población inicial inteligente...")
    population = [create_smart_individual() for _ in range(config['population_size'])]

    best_fitness = 0 #mejor fitness encontrado
    best_individual = None #mejor individuo encontrado
    fitness_history = [] #historial de fitness por generacion

    print("-- Empezando la evolución...")

    #bucle de evolución
    for gen in range(config['generations']): #Itera hasta 50 generaciones
        # Verifica si se ha alcanzado el límite de tiempo
        if time.time() - start_time > config['time_limit_seconds']:
            print(f"-- Límite de tiempo alcanzado en la generación {gen}")
            break

        # Evalua fitness de la población
        fitness_scores = [enhanced_fitness(ind) for ind in population] # calcula el fitness de cada individuo en la población

        # rastrea el mejor fitness de la generación
        gen_best = max(fitness_scores) # mejor fitness de la generación
        fitness_history.append(gen_best) # historial de fitness por generacion

        if gen_best > best_fitness: # Si es mejor que el mejor global
            best_fitness = gen_best
            best_individual = population[fitness_scores.index(gen_best)].copy() #guardamos una copia del mejor individuo encontrado

        # Informe de progreso
        if gen % 10 == 0 or gen == config['generations'] - 1: # cada 10 generaciones o en la última
            avg_fitness = np.mean(fitness_scores) #fitness promedio de la generación
            print(f"   Generacion {gen:2d}: Mejor ={gen_best:6.0f}, promedio={avg_fitness:6.0f}") #imprime el mejor y promedio fitness de la generación

        # Crear nueva generación
        new_population = []

        # elitismo - mantener los mejores individuos
        elite_indices = np.argsort(fitness_scores)[-config['elitism_size']:] #indices de los mejores individuos
        for idx in elite_indices:
            new_population.append(population[idx].copy()) #copia de los mejores individuos para la nueva generación

        # Generar descendencia
        while len(new_population) < config['population_size']: # Mientras la nueva población sea menor que el tamaño de la población
            parent1 = tournament_selection(population, fitness_scores) #selecciona el primer padre
            parent2 = tournament_selection(population, fitness_scores) #selecciona el segundo padre

            child1, child2 = smart_crossover(parent1, parent2) #cruza los padres para crear dos hijos
            child1 = smart_mutation(child1)
            child2 = smart_mutation(child2)
            # Añadir hijos a la nueva población
            new_population.extend([child1, child2])

        population = new_population[:config['population_size']] #limite a 25 individuos la nueva población

    # Analyze final result
    elapsed = time.time() - start_time # tiempo total de ejecución

    if best_individual is not None:
        selected_mask = best_individual.astype(bool) # crea una mascara booleana para los proyectos seleccionados
        selected_projects = df[selected_mask] #filtra los proyectos seleccionados usando la mascara booleana

        results = {
            'total_projects': len(selected_projects), #numero total de proyectos seleccionados
            'total_cost': float(selected_projects['MONTO_VIABLE'].sum()), #suma de montos viables de proyectos seleccionados
            'total_progress': float(selected_projects['AVANCE_FISICO'].sum()), #suma de avance fisico de proyectos seleccionados
            'total_beneficiaries': float(selected_projects['BENEFICIARIO'].sum()), #suma de beneficiarios de proyectos seleccionados
            'budget_utilization': float(selected_projects['MONTO_VIABLE'].sum() / config['max_budget']), #utilización del presupuesto
            'departments_covered': int(selected_projects['DEPARTAMENTO'].nunique()), #numero de departamentos cubiertos por proyectos seleccionados
            'sectors_covered': int(selected_projects['SECTOR'].nunique()),  #numero de sectores cubiertos por proyectos seleccionados
            'avg_efficiency': float(np.mean(efficiency[selected_mask])),    #eficiencia promedio de proyectos seleccionados
            'execution_time': elapsed,
            'generations_completed': gen + 1,
            'final_fitness': float(best_fitness)
        }

        print(f"\n-- GA mejorado completado en {elapsed:.1f} segundos")
        print(f"-- Resultados mejorados:")
        print(f"   Proyectos seleccionados: {results['total_projects']}")
        print(f"   Progreso total: {results['total_progress']:,.0f}")
        print(f"   Presupuesto utilizado: {results['budget_utilization']:.1%}")
        print(f"   Departmentos: {results['departments_covered']}")
        print(f"   Sectores: {results['sectors_covered']}")
        print(f"   Eficiencia promedio: {results['avg_efficiency']:.1f}")

        return results, selected_projects
    else:
        print("-- No se encontró ninguna solución")
        return None, None


In [20]:
def comprehensive_benchmarks(df: pd.DataFrame, max_budget: float, max_projects: int):
    """Ejecuta benchmarks completos para comparación"""

    print(f"\n-- PUNTOS DE REFERENCIA INTEGRALES")
    print("="*35)

    benchmarks = {} #diccionario para almacenar resultados de benchmarks

    # 1. Random Selection
    print("-- Punto de referencia de selección aleatoria...")
    random_results = []
    for run in range(10):  # Multiple runs for statistical validity
        sample = df.sample(n=min(max_projects, len(df)), random_state=42+run)
        cumulative_cost = sample['MONTO_VIABLE'].cumsum()
        feasible_mask = cumulative_cost <= max_budget

        if feasible_mask.any():
            feasible_sample = sample[feasible_mask]
            random_results.append({
                'progress': feasible_sample['AVANCE_FISICO'].sum(),
                'cost': feasible_sample['MONTO_VIABLE'].sum(),
                'projects': len(feasible_sample),
                'departments': feasible_sample['DEPARTAMENTO'].nunique()
            })

    if random_results:
        avg_random = {
            'method': 'Random Selection',
            'avg_progress': np.mean([r['progress'] for r in random_results]),
            'avg_cost': np.mean([r['cost'] for r in random_results]),
            'avg_projects': np.mean([r['projects'] for r in random_results]),
            'avg_departments': np.mean([r['departments'] for r in random_results]),
            'std_progress': np.std([r['progress'] for r in random_results])
        }
        benchmarks['random'] = avg_random
        print(f"   Progreso promedio: {avg_random['avg_progress']:,.0f} (±{avg_random['std_progress']:.0f})")

    # 2. Greedy by Efficiency
    print("-- Punto de referencia de eficiencia Greedy...")
    efficiency = df['AVANCE_FISICO'] / (df['MONTO_VIABLE'] / 1_000_000)
    efficiency = efficiency.replace([np.inf, -np.inf], 0).fillna(0)

    df_sorted = df.copy()
    df_sorted['efficiency'] = efficiency
    df_sorted = df_sorted.sort_values('efficiency', ascending=False)

    selected_indices = []
    current_cost = 0

    for idx, row in df_sorted.iterrows():
        if len(selected_indices) >= max_projects:
            break
        if current_cost + row['MONTO_VIABLE'] <= max_budget:
            selected_indices.append(idx)
            current_cost += row['MONTO_VIABLE']

    if selected_indices:
        greedy_sample = df.loc[selected_indices]
        greedy_results = {
            'method': 'Greedy Efficiency',
            'total_progress': greedy_sample['AVANCE_FISICO'].sum(),
            'total_cost': greedy_sample['MONTO_VIABLE'].sum(),
            'total_projects': len(greedy_sample),
            'departments_covered': greedy_sample['DEPARTAMENTO'].nunique(),
            'avg_efficiency': efficiency[selected_indices].mean()
        }
        benchmarks['greedy'] = greedy_results
        print(f"   Progreso total: {greedy_results['total_progress']:,.0f}")

    # 3. Greedy by Progress
    print("-- Punto de referencia del progreso codicioso...")
    df_by_progress = df.sort_values('AVANCE_FISICO', ascending=False)

    selected_indices = []
    current_cost = 0

    for idx, row in df_by_progress.iterrows():
        if len(selected_indices) >= max_projects:
            break
        if current_cost + row['MONTO_VIABLE'] <= max_budget:
            selected_indices.append(idx)
            current_cost += row['MONTO_VIABLE']

    if selected_indices:
        progress_greedy_sample = df.loc[selected_indices]
        progress_greedy_results = {
            'method': 'Greedy Progress',
            'total_progress': progress_greedy_sample['AVANCE_FISICO'].sum(),
            'total_cost': progress_greedy_sample['MONTO_VIABLE'].sum(),
            'total_projects': len(progress_greedy_sample),
            'departments_covered': progress_greedy_sample['DEPARTAMENTO'].nunique()
        }
        benchmarks['greedy_progress'] = progress_greedy_results
        print(f"   Total progress: {progress_greedy_results['total_progress']:,.0f}")

    return benchmarks


In [21]:
def complete_improved_workflow(input_file: str):
    """
    Complete improved workflow with better GA performance
    """

    print("-- FLUJO DE TRABAJO ULTRA RÁPIDO COMPLETAMENTE MEJORADO")
    print("="*45)
    print("Objetivo: Rendimiento superior de GA en menos de 5 minutos")

    total_start = time.time()

    # Step 1: Load or create sample (reuse if exists)
    sample_file = "ultra_fast_sample.csv"
    try:
        df_sample = pd.read_csv(sample_file, encoding='utf-8')
        print(f"-- Usando muestra existente: {len(df_sample):,} projectos")
    except:
        print("-- Creando nueva muestra...")
        df_large = pd.read_csv(input_file, encoding='utf-8')

        # Better sampling strategy
        df_large['temp_efficiency'] = df_large['AVANCE_FISICO'] / np.log10(df_large['MONTO_VIABLE'] + 1)
        df_large['temp_efficiency'] = df_large['temp_efficiency'].fillna(0)

        # 50% top performers + 50% stratified random
        top_1000 = df_large.nlargest(1000, 'temp_efficiency')
        remaining = df_large[~df_large.index.isin(top_1000.index)]

        # Stratified by department
        dept_samples = []
        for dept in remaining['DEPARTAMENTO'].unique():
            dept_df = remaining[remaining['DEPARTAMENTO'] == dept]
            sample_size = max(1, min(50, len(dept_df) // 100))
            dept_sample = dept_df.sample(n=sample_size, random_state=42)
            dept_samples.append(dept_sample)

        stratified_sample = pd.concat(dept_samples, ignore_index=True)
        df_sample = pd.concat([top_1000, stratified_sample], ignore_index=True)
        df_sample = df_sample.drop('temp_efficiency', axis=1)
        df_sample.to_csv(sample_file, index=False, encoding='utf-8')
        print(f"-- Nueva muestra creada: {len(df_sample):,} projectos")

    # Step 2: Run comprehensive benchmarks
    total_budget = df_sample['MONTO_VIABLE'].sum()
    max_budget = total_budget * 0.25
    max_projects = min(len(df_sample) // 8, 300)

    benchmarks = comprehensive_benchmarks(df_sample, max_budget, max_projects)

    # Step 3: Run improved GA
    print(f"\n🧬 Ejecutando GA mejorado...")
    ga_results, selected_projects = improved_ultra_fast_ga(sample_file, max_minutes=3)

    # Step 4: Compile final results
    total_elapsed = time.time() - total_start

    if ga_results:
        print(f"\n-- ¡FLUJO DE TRABAJO MEJORADO COMPLETADO!")
        print(f"-- Tiempo total: {total_elapsed:.1f} segundos")
        print(f"\n-- COMPARACIÓN FINAL:")

        if 'random' in benchmarks:
            print(f"   Seleccion Aleatoria: {benchmarks['random']['avg_progress']:,.0f} progreso promedio")

        if 'greedy' in benchmarks:
            print(f"    Eficiencia Greedy: {benchmarks['greedy']['total_progress']:,.0f} progreso total")

        if 'greedy_progress' in benchmarks:
            print(f"   Progreso Greedy: {benchmarks['greedy_progress']['total_progress']:,.0f} progreso total")

        print(f"   Optimizacion GA: {ga_results['total_progress']:,.0f} progreso total")

        # Calculate improvements
        if 'greedy' in benchmarks:
            improvement = (ga_results['total_progress'] / benchmarks['greedy']['total_progress'] - 1) * 100
            print(f"   Mejora de GA: +{improvement:.1f}% vs mejor greedy")

        # Save comprehensive results
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        results_file = f"comprehensive_ga_results_{timestamp}.json"

        final_results = {
            'dataset_info': {
                'sample_size': len(df_sample),
                'total_budget': float(total_budget),
                'optimization_budget': float(max_budget),
                'max_projects': max_projects
            },
            'benchmarks': benchmarks,
            'genetic_algorithm': ga_results,
            'execution_info': {
                'total_execution_time': total_elapsed,
                'timestamp': timestamp
            }
        }

        with open(results_file, 'w', encoding='utf-8') as f:
            json.dump(final_results, f, indent=2, ensure_ascii=False)

        print(f"\n-- Resultados completos guardados en: {results_file}")
        print(f"-- ¡TODO LISTO!")

        return final_results
    else:
        print("-- La optimización de GA falló")
        return None


In [22]:
if __name__ == "__main__":
    # Run improved workflow
    input_dataset = "dataset_GA_optimizado.csv"

    print("🚀 Starting improved ultra-fast workflow...")
    print("Target: Demonstrate GA superiority in under 5 minutes!")

    results = complete_improved_workflow(input_dataset)

🚀 Starting improved ultra-fast workflow...
Target: Demonstrate GA superiority in under 5 minutes!
-- FLUJO DE TRABAJO ULTRA RÁPIDO COMPLETAMENTE MEJORADO
Objetivo: Rendimiento superior de GA en menos de 5 minutos
-- Usando muestra existente: 2,000 projectos

-- PUNTOS DE REFERENCIA INTEGRALES
-- Punto de referencia de selección aleatoria...
   Progreso promedio: 9,901 (±583)
-- Punto de referencia de eficiencia Greedy...
   Progreso total: 24,893
-- Punto de referencia del progreso codicioso...
   Total progress: 25,000

🧬 Ejecutando GA mejorado...

--IMPLEMENTANDO ALGORITMO GENETICO (3 min max)
   --Configuración mejorada:
   Projectos: 2,000
   Presupuesto máximo: $979,910,892
   Proyectos máximos: 250
   Población: 25
   Generationes: 50
-- Creando una población inicial inteligente...
-- Empezando la evolución...
   Generacion  0: Mejor = 13547, promedio= 12221
   Generacion 10: Mejor = 24784, promedio= 24633
   Generacion 20: Mejor = 25112, promedio= 25087
   Generacion 30: Mejor =